In [2]:
import os # Interoperable file paths
import pathlib # Find the home folder
import geopandas as gpd # Work with vector data
import hvplot.pandas


In [3]:
# Define and create the project data directory
data_dir = os.path.join(
    # Home directory
    pathlib.Path.home(),
    # Earth analytics data directory
    'earth-analytics',
    'data',
    # Project directory
    'habitat-suitability',
)
# Create data directory in the home folder
os.makedirs(data_dir, exist_ok=True)

Define your study area:
- US Forest Service National Forests: George Washington National Forest and Jefferson National Forest
- Examining *Tsuga canadensis* (Eastern Hemlock)

In [ ]:
# Create site boundary data directory in the project folder
boundary_dir = os.path.join(data_dir, 'USFS')
os.makedirs(boundary_dir, exist_ok=True)

# Define info for USFS National Forests download
boundary_url = "https://data-usfs.hub.arcgis.com/api/download/v1/items/3451bcca1dbc45168ed0b3f54c6098d3/shapefile?layers=0"
boundary_path = os.path.join(boundary_dir, 'USFS_boundary.shp')

# Only download once
if not os.path.exists(boundary_path):
    boundary_gdf = gpd.read_file(boundary_url)
    boundary_gdf.to_file(boundary_path)

# Load from file
else:
    boundary_gdf = gpd.read_file(boundary_path)

# Check the data
boundary_gdf

,OBJECTID,PROCLAIMED,FORESTNAME,GIS_ACRES,SHAPEAREA,SHAPELEN,geometry
0,197651,66329010328,Bighorn National Forest,1112645.655,0.509899,4.943363,"POLYGON ((-107.54297 44.93778, -107.54297 44.9..."
1,197652,93007010328,Gifford Pinchot National Forest,1532172.644,0.722290,11.679336,"MULTIPOLYGON (((-122.27146 45.75118, -122.2711..."
2,197653,96812010328,Manti-La Sal National Forest,1337654.214,0.561732,9.953308,"MULTIPOLYGON (((-111.40255 39.9808, -111.40093..."
3,197654,96813010328,Uinta National Forest,961743.024,0.411377,6.874785,"MULTIPOLYGON (((-111.5513 40.59377, -111.5513 ..."
4,197655,105935010328,Kaibab National Forest,1601002.978,0.647156,9.140002,"MULTIPOLYGON (((-112.06365 36.8781, -112.06365..."
...,...,...,...,...,...,...,...
149,198024,295501010328,Grand Mesa National Forest,351209.089,0.147989,4.076972,"MULTIPOLYGON (((-107.8139 39.37261, -107.8136 ..."
150,198025,295502010328,Arapaho National Forest,767577.662,0.327520,9.984158,"MULTIPOLYGON (((-105.82719 40.24581, -105.8277..."
151,198026,295503010328,Stanislaus National Forest,1092025.607,0.454330,6.241501,"MULTIPOLYGON (((-120.40147 38.30669, -120.4014..."
152,198027,295504010328,Tahoe National Forest,1225633.350,0.518547,10.476720,"MULTIPOLYGON (((-121.14404 39.463, -121.13938 ..."


In [14]:
# Select data of the two forests of interest
GWNF_gdf = boundary_gdf.loc[boundary_gdf['FORESTNAME'] == 'George Washington National Forest']
JNF_gdf = boundary_gdf.loc[boundary_gdf['FORESTNAME'] == 'Jefferson National Forest']

# Create plots to visualize the boundaries
GWNF_plot = (
    GWNF_gdf.dissolve().hvplot(
        geo=True, tiles='EsriImagery',
        title="George Washington National Forest",
        fill_color=None, line_color ='white', line_width=3,
        frame_width=600
    )
)

JNF_plot = (
    JNF_gdf.dissolve().hvplot(
        geo=True, tiles='EsriImagery',
        title="Jefferson National Forest",
        fill_color=None, line_color ='orange', line_width=3,
        frame_width=600
    )
)

(
    GWNF_plot
+
    JNF_plot
)

:Layout
   .Overlay.I  :Overlay
      .WMTS.I     :WMTS   [Longitude,Latitude]
      .Polygons.I :Polygons   [Longitude,Latitude]
   .Overlay.II :Overlay
      .WMTS.I     :WMTS   [Longitude,Latitude]
      .Polygons.I :Polygons   [Longitude,Latitude]

In [10]:
%store data_dir GWNF_gdf JNF_gdf

Stored 'data_dir' (str)
Stored 'GWNF_gdf' (GeoDataFrame)
Stored 'JNF_gdf' (GeoDataFrame)


Calculate at least one derived topographic variable (slope or aspect) to use in your model. You probably will wish to use the xarray-spatial library, which is available in the latest earth-analytics-python environment (but will need to be installed/updated if you are working on your own machine). Note that calculated slope may not be correct if you are using a CRS with units of degrees; you should re-project into a projected coordinate system with units of meters, such as the appropriate UTM Zone.

Harmonize your data - make sure that the grids for each of your layers match up. Check out the ds.rio.reproject_match() method from rioxarray.

Build your model. You can use any model you wish, so long as you explain your choice. However, if you are not sure what to do, we recommend building a fuzzy logic model (see below).

Present your results in at least one figure for each grassland/climate scenario combination.